In [225]:
import matplotlib.pyplot as plt
from retrieval import MovielensModel
import numpy as np
import tensorflow as tf
plt.style.use('seaborn-whitegrid')
from keras.utils.vis_utils import plot_model

In [167]:
import pandas as pd

In [168]:
from collections import OrderedDict

In [204]:
ratings_train = tf.data.experimental.CsvDataset('../datasets/v5/ratings_train.csv',
                                                header = True,
                                                record_defaults = [tf.string, tf.string, tf.float32])\
                                                    .map(lambda x1, x2, x3: OrderedDict({'user_id' : x1, 'movie_title' :x2}))

ratings_test = tf.data.experimental.CsvDataset('../datasets/v5/ratings_test.csv',
                                                header = True,
                                                record_defaults = [tf.string, tf.string, tf.float32])\
                                                    .map(lambda x1, x2, x3: OrderedDict({'user_id' : x1, 'movie_title' :x2}))

ratings_valid = tf.data.experimental.CsvDataset('../datasets/v5/ratings_valid.csv',
                                                header = True,
                                                record_defaults = [tf.string, tf.string, tf.float32])\
                                                    .map(lambda x1, x2, x3: OrderedDict({'user_id' : x1, 'movie_title' :x2}))

movies = tf.data.experimental.CsvDataset('../datasets/v5/movies.csv',
                                         header = True,
                                         record_defaults = [tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.string, tf.string])\
                                            .map(lambda x1, x2, x3, x4, x5, x6, x7: OrderedDict({'movie_title' : x6}))


In [206]:
unique_movie_titles = np.unique(np.concatenate(list(movies.map(lambda x: x['movie_title']).batch(1000))))

In [207]:
unique_movie_titles.shape

(27262,)

In [208]:
unique_user_ids = np.unique(np.concatenate(list(ratings_train.map(lambda x: x["user_id"]).batch(1000))))

In [209]:
unique_user_ids[0]

b'100016'

In [210]:
len(unique_user_ids)

6925

In [211]:
ratings_train_cached = ratings_train.shuffle(100000).batch(2048).cache()
ratings_test_cached = ratings_test.batch(4096).cache()

In [244]:
model = MovielensModel([16, 8], movies.map(lambda x: x['movie_title']), unique_user_ids, unique_movie_titles)
model.compile(optimizer=tf.keras.optimizers.Adam())

In [245]:
import datetime

In [249]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=5)

In [250]:
num_epochs = 10

In [251]:
two_layer_history = model.fit(
    ratings_train_cached,
    validation_data=ratings_test_cached,
    validation_freq=5,
    epochs=num_epochs,
    verbose=1,
          callbacks=[tensorboard_callback])

Epoch 1/10
    478/Unknown - 23s 49ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 14991.7161 - regularization_loss: 0.0000e+00 - total_loss: 14991.7161

AttributeError: 'VocabWeightHandler' object has no attribute 'name'

In [223]:
accuracy = two_layer_history.history["val_factorized_top_k/top_100_categorical_accuracy"][-1]
print(f"Top-100 accuracy: {accuracy:.2f}.")

Top-100 accuracy: 0.15.


In [253]:
two_layer_history.history

{'factorized_top_k/top_1_categorical_accuracy': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'factorized_top_k/top_5_categorical_accuracy': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'factorized_top_k/top_10_categorical_accuracy': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'factorized_top_k/top_50_categorical_accuracy': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'factorized_top_k/top_100_categorical_accuracy': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'loss': [7400.9580078125,
  7285.525390625,
  7196.15185546875,
  7143.38232421875,
  7081.58642578125,
  6998.41650390625,
  6931.2392578125,
  6868.03076171875,
  6807.6201171875,
  6764.88623046875],
 'regularization_loss': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'total_loss': [7400.9580078125,
  7285.525390625,
  7196.15185546875,
  7143.38232421875,
  7081.58642578125,
  6998.41650390625,
  6931.23925781

In [232]:
model(ratings_test_cached,)










NotImplementedError: Exception encountered when calling layer "movielens_model_2" (type MovielensModel).

Unimplemented `tf.keras.Model.call()`: if you intend to create a `Model` with the Functional API, please provide `inputs` and `outputs` arguments. Otherwise, subclass `Model` with an overridden `call()` method.

Call arguments received:
  • inputs=<CacheDataset element_spec=OrderedDict([('user_id', TensorSpec(shape=(None,), dtype=tf.string, name=None)), ('movie_title', TensorSpec(shape=(None,), dtype=tf.string, name=None))])>
  • training=None
  • mask=None

In [ ]:
m2 = tf.saved_model.load('/content/drive/MyDrive/models/model_ret_pkl')